In [950]:
import inspect
import random
from weakref import WeakKeyDictionary

In [951]:
class Symbol(object):
    def __init__(self, values):
        self.parent = None
        self.InitVals(values)
        
    def InitVals(self, values):
        # set all of my potential values
        
        # allow synyax shorthand 
        # symbol = Symbol(Fruit) = Symbol([Apple, Pear])
        if inspect.isclass(values):
            self.values = []
            for subclass in values.__subclasses__():
                self.values.append(subclass())
        else:
            # TODO: ensure iterable
            self.values = values
        
        # initialize state to superposition
        self._value = None
        
        # expose all of my possible values' attributes as if they were my own
        self.CalcAttributes()
        
    def __repr__(self):
        return str(self.val)
    
    def __getattr__(self, attr):
        
        # allow self.val to inspect without collapsing my value
        if attr == 'val':
            if self._value:
                return self._value
            else:
                
                if len(self.values) == 1:
                    return self.values[0]
                else:
                    return self.values
        raise AttributeError("{} object has no attribute {}".format(self.__class__, attr))
    
    def Observe(self):
        if self._value:
            return self._value
        else:
            return self.Collapse()
    
    def Collapse(self, new_value=None):
        
        # set value
        if new_value:
            
            # TODO: restrict to all values consistent with new_value
            # if new_value is array
            # if new_value is symbol
            # if new_value is object / type
            # if new_value is primitive
            
            # we do not need to fully collapse
            
            self._value = new_value
        else:
            self._value = random.choice(self.values)
            # now I only have one value
            self.values = [self._value]
        
        # propagate the change upwards: everything above me must be consistent
        # with me having this value.
        if self.parent:
            self.parent.Restrict(self, self._value)

        # update my attributes
        self.CalcAttributes()
        
        return self._value
    
    def CalcAttributes(self):
        
        # expose the attributes of all my potential values
        attribute_values = {}
        for obj in self.values:
            
            # list all of the object's public values
            attributes = [i for i in dir(obj) if i[0] != '_' and not callable(getattr(obj, i))]
            
            # add all of obj.attribute to the set of possible values
            for attribute in attributes:
                
                # initialize dict
                if attribute not in attribute_values:
                    attribute_values[attribute] = []
                
                # get obj.val
                val = obj.__getattribute__(attribute)
                
                # if obj.val is a symbol, i must be consistent with all of its values
                if isinstance(val, Symbol):
                    attribute_values[attribute].extend(val.values)
                else:
                    # TODO: check if iterable
                    attribute_values[attribute].append(val)
        
        # add these attributes to myself.
        for key, vals in attribute_values.items():
                symbol = Symbol(vals)
                symbol.parent = self
                
                # TODO: check if i already have the attribute first
                setattr(self, key, symbol)

    def Restrict(self, child, newval):
        # change content of values to be consistent with new value for child
        # Restriction is:
        # now that we know the specific value of this property, which of my values are consistent with that?
        
        # determine the attribute
        for key, val in self.__dict__.items():
            if val == child:
                attr = key
                break
        
        # check each of my potential values to see if they are consistent with having
        # obj.attr = newval
        new_values = []
        for obj in self.values:
            
            # if the object doesnt even have this attribute, forget it
            if not hasattr(obj, attr):
                continue
            
            # get the object's obj.attr
            obj_attr_val =  getattr(obj, attr)
            
            # if is obj.attr is symbol, how can we make sure it is consistent
            # with obj.attr = newval?
            
            # if newval is symbol
            #    if obj.attr is 
            # if newval is array
            # if newval is primitive
            # 
            
            if isinstance(obj_attr_val, Symbol):
                
                # obj.attr = Symbol ∩ newval
                # this must 
                if newval in obj_attr_val.values:
                    obj_attr_val.Collapse(new_value=newval)
                    new_values.append(obj)
                    continue
            else:
                # if obj.attr is not symbol, check that obj.attr is consistent with newval
                
                if obj_attr_val == newval:
                    new_values.append(obj)
                    continue
        
        # big problem here if true
        if len(new_values) == 0:
            raise Exception('empty restriction!')
            
        self.values = new_values
        
        # if we've restricted to a single value, might as well collapse now.
        if len(new_values) == 1:
            self.Collapse()
            

# collapse restricts self.values
when we are told to collapse to new_val, we must restrict
self.values to be only those values that are consistent with
new_val.

"Ensure that self.value has to be new_val"

value -> new_val:
for each value v in values:
    if new_val ⊆ v
    then include v with v -> v ∩ new_val
        
        
# restrict restricts self.values
this happens when the symbolic attribute of self collapses or restricts.
now we must restrict self.values to only those values who have the 
attribute, and that attribute is consistent with new_val.

"Ensure that self.attribute has to be new_val"

self.attr -> new_val
for each value v in values:
    if
    v has attribute attr
    and
    new_val.attr ⊆ v.attr
    then include v with v.attr -> v.attr ∩ new_val.attr


# define nv ⊆ v
       
either can be literal, symbolic, array, or type.

v ∩ nv ≠ Ø:

define x ∩ y for x, y ∈ literal, symbolic, array, type

v is a superclass or class of nv
v is an iterable containing at least one nv

v -> collapse(v, nv)


define restriction here, too. this is the point of recursion
for each attribute?

for each attribute nv.a, nv has nv.a and v.a ∩ nv.a ≠ Ø

and recursively...?



# unification

how to unify collapse and restrict? they are the same, except for the 
v has attr line, which hints at the proper way to handle this recursively.

# recursion

What if an attribute is buried two levels deep?
self.attr1.attr2 collapses to attr2.values
now attr1 -> restriction(attr2, attr2.values)
now self -> restriction(attr1, attr1.values)

# order of tree traversal

Do we do one pass down then one pass up? simultaneous? which order?


Consistency requires that for each attr in Person,
1. Person.attr == Attacker.attr, or
2. Person.attr contains Attacker.attr

e.g.
1. Person.item = weapon, Attacker.item = weapon
1.a. Person.Item = Weapon, Attacker.item = Weapon'

2. Person.item = Item, Attacker.attr = weapon
2.a. Person.item = Item, Attacker.item = Weapon






we restrict to self.values -> self.values ∩ new_value.values



In [952]:
class Symbolic(object):
    def __init__(self):
        self.symbols = WeakKeyDictionary()
 
    def __get__(self, instance_obj, objtype):
#         return self.symbols[instance_obj].Ref()
        return self.symbols[instance_obj]
 
    def __set__(self, instance, values):
        self.symbols[instance] = Symbol(values)
 
    def __delete__(self, instance):
        del self.symbols[instance]

        
class Fruit(object):
    color = Symbolic()
    def __init__(self, name):
        self.name = name
    def method(self):
        print('method man')

class Apple(Fruit):
    def __init__(self):
        self.name = 'Apple'
        self.color = ['red', 'green']

class Pear(Fruit):
    def __init__(self):
        self.name = 'Pear'
        self.color = ['red', 'yellow']

        
class State(object):
    fruit = Symbolic()
    def __init__(self):
        self.fruit = Fruit

In [1001]:
state = State()

In [1004]:
state.fruit.Observe()

In [1005]:
state.fruit.color

['red', 'yellow']

# Factories

Ensuring / requiring / generating:

"we need a bad guy in this scene. Make it happen"

In the present system, this means that if there is an uncollapsed Person in the scene, I can collapse that person to an Attacker.
Attacker knows it wants to attack player, so it moves in to strike.
Once the player sees the Attacker, it is observed, and collapses into Ninja.

What about if there is no Person or uncollapsed Person in the scene?

The scene also defines various ways that objects can be created. Object factories exist:
A window is an object factory for a message (note on brick), people (climb in window)

A computer is an object factory for a message (email, video hack)

A door is an object factory for people

A container is an object factory for items

---

Once the director issues a directive, it is up to the state to enact it. 
First the state searches all objects in superposition to see if it can collapse an object to the required one.
If not, then the state searches its available factories, and finds if any factory provides a type that is superclass of the requested item.

In [896]:
state.fruit